In [ ]:
import os
import pandas as pd

import torch
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

In [ ]:
DATA_DIR = "data"
DATA_FILENAME = "diamonds.csv"
data_path = os.path.join(DATA_DIR, DATA_FILENAME)

In [ ]:
df = pd.read_csv(data_path, index_col=0)
df

In [ ]:
df = pd.read_csv(data_path, index_col=0)
y_data = df.pop("price").values

In [ ]:
categoray_cols = ['cut', "color", "clarity"]
for col_name in categoray_cols:
    df[col_name] = pd.Categorical(df[col_name])
df_onehot = pd.get_dummies(df)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df_onehot)
df_normalize = scaler.transform(df_onehot)

In [ ]:
X_data = df_normalize.copy()

In [ ]:
X_data.shape, y_data.shape

In [ ]:
class LinearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(
            in_features=inputSize, out_features=outputSize, bias=True)

    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
X_data[0].shape[0]

In [ ]:
number_of_data, number_of_features = X_data.shape
number_of_data  = y_data.shape

In [ ]:
number_of_features

In [ ]:
number_of_data

In [ ]:
model = LinearRegression(number_of_features, 1)

if torch.cuda.is_available():
    model.cuda()

In [ ]:
learningRate = 0.1 
epochs = 10000

criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.3, 
    random_state=42, shuffle=True)


In [ ]:
loss_values = []
rmse_values = []

for epoch in range(epochs):
    # Converting inputs and labels to Variable
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(X_train).float().cuda())
        labels = Variable(torch.from_numpy(y_train).float().cuda())
    else:
        inputs = Variable(torch.from_numpy(X_train))
        labels = Variable(torch.from_numpy(y_train))

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion(outputs, labels)
    
    
    if epoch % 100 == 0:
        with torch.no_grad(): # we don't need gradients in the testing phase
            if torch.cuda.is_available():
                y_pred = model(Variable(
                    torch.from_numpy(X_test).float().cuda())).cpu().data.numpy()
            else:
                y_pred = model(
                    Variable(torch.from_numpy(X_test))).data.numpy()
            RMSE = mean_squared_error(y_test, y_pred.reshape(-1)) ** 0.5

        print(loss)
        print('epoch {}, loss {}, rmse {}'.format(
            epoch, loss.item(), RMSE))
        loss_values.append(loss)
        rmse_values.append(RMSE)
    
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

   

In [ ]:

plt.plot(loss_values)


In [ ]:
with torch.no_grad(): # we don't need gradients in the testing phase
    if torch.cuda.is_available():
        predicted = model(Variable(
            torch.from_numpy(X_train).float().cuda())).cpu().data.numpy()
    else:
        predicted = model(
            Variable(torch.from_numpy(X_train))).data.numpy()
    print(predicted)

In [ ]:
with torch.no_grad(): # we don't need gradients in the testing phase
    if torch.cuda.is_available():
        y_pred = model(Variable(
            torch.from_numpy(X_test).float().cuda())).cpu().data.numpy()
    else:
        y_pred = model(
            Variable(torch.from_numpy(X_test))).data.numpy()
    print(y_pred)

In [ ]:
fig,ax = plt.subplots()
ax.plot(loss_values, color="red", marker="o")
ax2=ax.twinx()
ax2.plot(rmse_values,color="blue",marker="o")
plt.show()

In [ ]:
RMSE = mean_squared_error(y_test, y_pred.reshape(-1)) ** 0.5
RMSE